# NIFTY Market Data Engine — API Test Suite
**Version:** 3.0  
**Author:** Data Pipeline Team  
**Purpose:** Validate all query methods of `NiftyMarketData` and demonstrate correct usage patterns for the Pricing, Hedging, and Volatility Surface teams.

---

## Table of Contents
1. [Environment Setup](#1-environment-setup)
2. [Engine Initialisation](#2-engine-initialisation)
3. [Discovery Queries](#3-discovery-queries)
4. [Basic Option Chain Query](#4-basic-option-chain-query)
5. [Strike Filtering](#5-strike-filtering)
6. [Option Type Filtering (Calls / Puts)](#6-option-type-filtering)
7. [Intraday Time Window Query](#7-intraday-time-window-query)
8. [Liquidity Filtering](#8-liquidity-filtering)
9. [Combined Query (All Filters Together)](#9-combined-query)
10. [ATM Strike Grid Generation](#10-atm-strike-grid-generation)
11. [Spot Price Merge Validation](#11-spot-price-merge-validation)
12. [Volatility Surface Snapshot](#12-volatility-surface-snapshot)
13. [Time Series Query (Multi-Day)](#13-time-series-query)
14. [Error Handling Demonstrations](#14-error-handling)
15. [Performance Benchmarks](#15-performance-benchmarks)
16. [Cache Management](#16-cache-management)

---
## 1. Environment Setup

In [ ]:
import sys
import os
import time
import pandas as pd

print("Python executable:", sys.executable)
print("Working directory:", os.getcwd())
print("Pandas version   :", pd.__version__)

Python executable: c:\MyOneDrive\OneDrive\QuantFin\NIFTY_DATA_PIPELINE\venv\Scripts\python.exe
Working directory: c:\MyOneDrive\OneDrive\QuantFin\NIFTY_DATA_PIPELINE\nifty-market-data-engine\notebooks
Pandas version   : 2.3.3


In [ ]:
# Add project root to Python path
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
if PROJECT_ROOT not in sys.path:
    sys.path.append(PROJECT_ROOT)

print("Project root added:", PROJECT_ROOT)

Project root added: c:\MyOneDrive\OneDrive\QuantFin\NIFTY_DATA_PIPELINE\nifty-market-data-engine


---
## 2. Engine Initialisation
Set `BASE_DIR` to the root of the shared Google Drive dataset folder. All other paths are resolved automatically.

In [9]:
from api.marketdatav3 import NiftyMarketData

# ─── Set your local path to the shared dataset root ───────────────────────────
# BASE_DIR = r"D:\QuantFin\NiftyHistorical2024\raw_kaggle"
BASE_DIR = r"G:\.shortcut-targets-by-id\1f6XlJFCOVmETxGoJjD4O9WSmhB38IsQy\FinanceProject_LogicLabs\NiftyHistorical2024\raw_kaggle"
# ──────────────────────────────────────────────────────────────────────────────

md = NiftyMarketData(base_dir=BASE_DIR)

[NiftyMarketData] Engine initialised. Base directory: G:\.shortcut-targets-by-id\1f6XlJFCOVmETxGoJjD4O9WSmhB38IsQy\FinanceProject_LogicLabs\NiftyHistorical2024\raw_kaggle


---
## 3. Discovery Queries
Use these before any data query to confirm that the expiry, trade date, and strikes you want actually exist in the dataset.

### 3a. List available expiries for a trade date

In [10]:
expiries = md.list_expiries(trade_date="01JAN24")

print(f"Expiries available on 01 Jan 2024: {len(expiries)} found")
for e in expiries:
    print(" ", e)

Expiries available on 01 Jan 2024: 8 found
  01FEB24
  04JAN24
  11JAN24
  18JAN24
  25JAN24
  26DEC24
  27JUN24
  28MAR24


### 3b. List available trading days for a month

In [11]:
trading_days = md.list_trading_days(year=2024, month="JAN")

print(f"Trading days in January 2024: {len(trading_days)} found")
print(trading_days)

Trading days in January 2024: 21 found
['01JAN24', '02JAN24', '03JAN24', '04JAN24', '05JAN24', '08JAN24', '09JAN24', '10JAN24', '11JAN24', '12JAN24', '15JAN24', '16JAN24', '17JAN24', '18JAN24', '19JAN24', '23JAN24', '24JAN24', '25JAN24', '29JAN24', '30JAN24', '31JAN24']


### 3c. List available strikes for a specific expiry

In [12]:
strikes_available = md.list_strikes(expiry="01FEB24", trade_date="01JAN24")

print(f"Total strikes available: {len(strikes_available)}")
print("First 15 strikes:", strikes_available[:15])

Total strikes available: 55
First 15 strikes: [19850, 20000, 20100, 20150, 20200, 20300, 20500, 20700, 20950, 21000, 21050, 21100, 21150, 21200, 21250]


---
## 4. Basic Option Chain Query
Fetch the complete option chain for one expiry on one trade date. No filters applied — all strikes, both Calls and Puts, all intraday timestamps.

In [13]:
df = md.query_options(
    expiry="01FEB24",
    trade_date="01JAN24"
)

print(f"Rows returned : {len(df):,}")
print(f"Columns       : {df.columns.tolist()}")
print(f"Time range    : {df['timestamp'].min()} → {df['timestamp'].max()}")
print(f"Strikes range : {df['strike'].min()} → {df['strike'].max()}")
df.head()

Rows returned : 16,892
Columns       : ['timestamp', 'expiry_date', 'days_to_expiry', 'strike', 'option_type', 'open_price', 'high_price', 'low_price', 'close_price', 'market_price', 'volume', 'open_interest', 'spot_price']
Time range    : 2024-01-01 09:15:00 → 2024-01-01 15:29:00
Strikes range : 19850 → 23300


,timestamp,expiry_date,days_to_expiry,strike,option_type,open_price,high_price,low_price,close_price,market_price,volume,open_interest,spot_price
0,2024-01-01 09:15:00,2024-02-01,31,21700,P,287.15,287.15,271.60,271.60,271.60,250,6600,21710.40
1,2024-01-01 09:15:00,2024-02-01,31,23000,C,51.20,51.25,51.20,51.25,51.25,150,14100,21710.40
2,2024-01-01 09:16:00,2024-02-01,31,21750,C,410.15,410.15,410.15,410.15,410.15,50,300,21695.35
3,2024-01-01 09:16:00,2024-02-01,31,21700,P,290.60,290.60,290.60,290.60,290.60,150,6600,21695.35
4,2024-01-01 09:16:00,2024-02-01,31,23000,C,41.40,44.85,41.40,44.85,44.85,300,14100,21695.35


---
## 5. Strike Filtering
Request data for a specific list of strikes only.

In [14]:
df_strikes = md.query_options(
    expiry="01FEB24",
    trade_date="01JAN24",
    strikes=[21500, 21600, 21700, 21800, 21900]
)

print(f"Rows returned          : {len(df_strikes):,}")
print(f"Unique strikes returned: {sorted(df_strikes['strike'].unique())}")
df_strikes.head()

Rows returned          : 3,488
Unique strikes returned: [np.int64(21500), np.int64(21600), np.int64(21700), np.int64(21800), np.int64(21900)]


,timestamp,expiry_date,days_to_expiry,strike,option_type,open_price,high_price,low_price,close_price,market_price,volume,open_interest,spot_price
0,2024-01-01 09:15:00,2024-02-01,31,21700,P,287.15,287.15,271.60,271.60,271.60,250,6600,21710.40
1,2024-01-01 09:16:00,2024-02-01,31,21700,P,290.60,290.60,290.60,290.60,290.60,150,6600,21695.35
2,2024-01-01 09:17:00,2024-02-01,31,21500,C,590.05,590.05,590.05,590.05,590.05,50,1000,21709.55
3,2024-01-01 09:18:00,2024-02-01,31,21500,P,210.10,210.10,210.10,210.10,210.10,50,35100,21701.60
4,2024-01-01 09:18:00,2024-02-01,31,21700,P,285.15,285.15,285.15,285.15,285.15,100,6750,21701.60


---
## 6. Option Type Filtering
Filter to Calls only, or Puts only.

### 6a. Calls only

In [15]:
df_calls = md.query_options(
    expiry="01FEB24",
    trade_date="01JAN24",
    option_type="C"
)

print(f"Rows returned         : {len(df_calls):,}")
print(f"Option types in result: {df_calls['option_type'].unique()}")
df_calls.head()

Rows returned         : 10,548
Option types in result: ['C']


,timestamp,expiry_date,days_to_expiry,strike,option_type,open_price,high_price,low_price,close_price,market_price,volume,open_interest,spot_price
0,2024-01-01 09:15:00,2024-02-01,31,23000,C,51.20,51.25,51.20,51.25,51.25,150,14100,21710.40
1,2024-01-01 09:16:00,2024-02-01,31,21750,C,410.15,410.15,410.15,410.15,410.15,50,300,21695.35
2,2024-01-01 09:16:00,2024-02-01,31,23300,C,24.50,24.50,24.30,24.30,24.30,200,3850,21695.35
3,2024-01-01 09:16:00,2024-02-01,31,23000,C,41.40,44.85,41.40,44.85,44.85,300,14100,21695.35
4,2024-01-01 09:16:00,2024-02-01,31,22000,C,297.25,297.30,290.00,294.95,294.95,300,21700,21695.35


### 6b. Puts only

In [16]:
df_puts = md.query_options(
    expiry="01FEB24",
    trade_date="01JAN24",
    option_type="P"
)

print(f"Rows returned         : {len(df_puts):,}")
print(f"Option types in result: {df_puts['option_type'].unique()}")

Rows returned         : 6,344
Option types in result: ['P']


---
## 7. Intraday Time Window Query
Restrict data to a specific intraday window. NIFTY trades 09:15–15:30 IST.

In [17]:
df_window = md.query_options(
    expiry="01FEB24",
    trade_date="01JAN24",
    start="2024-01-01 09:30",
    end="2024-01-01 10:30"
)

print(f"Rows returned : {len(df_window):,}")
print(f"Earliest time : {df_window['timestamp'].min()}")
print(f"Latest time   : {df_window['timestamp'].max()}")
df_window.head()

Rows returned : 2,649
Earliest time : 2024-01-01 09:30:00
Latest time   : 2024-01-01 10:30:00


,timestamp,expiry_date,days_to_expiry,strike,option_type,open_price,high_price,low_price,close_price,market_price,volume,open_interest,spot_price
0,2024-01-01 09:30:00,2024-02-01,31,21700,C,478.40,478.40,478.40,478.40,478.40,0,0,21695.35
1,2024-01-01 09:30:00,2024-02-01,31,21700,P,284.35,284.35,284.35,284.35,284.35,0,6850,21695.35
2,2024-01-01 09:30:00,2024-02-01,31,21600,P,240.00,240.00,240.00,240.00,240.00,0,0,21695.35
3,2024-01-01 09:30:00,2024-02-01,31,21300,C,776.65,776.65,776.65,776.65,776.65,0,0,21695.35
4,2024-01-01 09:30:00,2024-02-01,31,21500,P,214.60,214.60,214.60,214.60,214.60,0,35100,21695.35


### 7b. Single-minute snapshot

In [18]:
df_snapshot = md.query_options(
    expiry="01FEB24",
    trade_date="01JAN24",
    start="2024-01-01 10:00",
    end="2024-01-01 10:00"
)

print(f"Rows at exactly 10:00 AM: {len(df_snapshot)}")
print(f"Unique timestamps       : {df_snapshot['timestamp'].unique()}")
df_snapshot[["strike", "option_type", "market_price", "spot_price"]].head(10)

Rows at exactly 10:00 AM: 44
Unique timestamps       : <DatetimeArray>
['2024-01-01 10:00:00']
Length: 1, dtype: datetime64[ns]


,strike,option_type,market_price,spot_price
0,21000,C,1089.95,21724.45
1,23200,C,25.45,21724.45
2,23250,C,19.40,21724.45
3,20500,P,41.75,21724.45
4,20950,P,156.70,21724.45
5,21000,P,97.00,21724.45
6,21050,P,130.00,21724.45
7,21100,P,150.00,21724.45
8,21150,P,157.00,21724.45
9,23000,C,42.60,21724.45


---
## 8. Liquidity Filtering
Many rows in this dataset have zero traded volume. Apply `min_volume` to retain only actively traded contracts — essential for accurate implied volatility estimation.

In [19]:
# Unfiltered baseline
df_all     = md.query_options(expiry="01FEB24", trade_date="01JAN24")
zero_vol   = (df_all["volume"] == 0).sum()
print(f"Total rows           : {len(df_all):,}")
print(f"Rows with zero volume: {zero_vol:,}  ({100*zero_vol/len(df_all):.1f}%)")

Total rows           : 16,892
Rows with zero volume: 15,648  (92.6%)


In [20]:
# Apply liquidity filter
df_liquid = md.query_options(
    expiry="01FEB24",
    trade_date="01JAN24",
    min_volume=10
)

print(f"Rows after min_volume=10 filter: {len(df_liquid):,}")
print(f"Minimum volume in result       : {df_liquid['volume'].min()}")
df_liquid.head()

Rows after min_volume=10 filter: 1,244
Minimum volume in result       : 50


,timestamp,expiry_date,days_to_expiry,strike,option_type,open_price,high_price,low_price,close_price,market_price,volume,open_interest,spot_price
0,2024-01-01 09:15:00,2024-02-01,31,21700,P,287.15,287.15,271.60,271.60,271.60,250,6600,21710.40
1,2024-01-01 09:15:00,2024-02-01,31,23000,C,51.20,51.25,51.20,51.25,51.25,150,14100,21710.40
2,2024-01-01 09:16:00,2024-02-01,31,22000,C,297.25,297.30,290.00,294.95,294.95,300,21700,21695.35
3,2024-01-01 09:16:00,2024-02-01,31,23300,C,24.50,24.50,24.30,24.30,24.30,200,3850,21695.35
4,2024-01-01 09:16:00,2024-02-01,31,21750,C,410.15,410.15,410.15,410.15,410.15,50,300,21695.35


---
## 9. Combined Query
All filters applied simultaneously: specific strikes, Calls only, intraday window, liquidity threshold.

This is the recommended pattern for the Pricing Team when computing implied volatility.

In [21]:
# First, find the ATM strikes dynamically
atm, grid = md.get_atm_strikes("01FEB24", "01JAN24", n_strikes=3, step=100)
print(f"ATM: {atm}")
print(f"Selected strikes: {grid}")

ATM: 21700
Selected strikes: [21400, 21500, 21600, 21700, 21800, 21900, 22000]


In [22]:
df_combined = md.query_options(
    expiry="01FEB24",
    trade_date="01JAN24",
    strikes=grid,
    option_type="C",
    start="2024-01-01 10:00",
    end="2024-01-01 12:00",
    min_volume=5
)

print(f"Rows returned : {len(df_combined):,}")
df_combined[["timestamp", "strike", "option_type", "market_price", "volume", "spot_price"]].head(10)

Rows returned : 52


,timestamp,strike,option_type,market_price,volume,spot_price
0,2024-01-01 10:00:00,21500,C,607.15,50,21724.45
1,2024-01-01 10:00:00,21800,C,408.00,50,21724.45
2,2024-01-01 10:03:00,21700,C,471.95,50,21725.90
3,2024-01-01 10:05:00,22000,C,299.50,450,21722.70
4,2024-01-01 10:05:00,21500,C,595.80,50,21722.70
5,2024-01-01 10:12:00,22000,C,291.55,100,21704.75
6,2024-01-01 10:13:00,21700,C,450.55,250,21706.80
7,2024-01-01 10:14:00,22000,C,292.00,50,21709.65
8,2024-01-01 10:15:00,22000,C,294.55,100,21712.75
9,2024-01-01 10:16:00,21500,C,587.60,100,21708.45


---
## 10. ATM Strike Grid Generation
Generate a symmetric grid of strikes around the at-the-money level, derived automatically from the opening spot price.

In [23]:
# Default: 10 strikes each side of ATM, step=100
atm, grid = md.get_atm_strikes(
    expiry="01FEB24",
    trade_date="01JAN24",
    n_strikes=10,
    step=100
)

print(f"ATM Strike     : {atm}")
print(f"Grid size      : {len(grid)} strikes")
print(f"Strike range   : {min(grid)} → {max(grid)}")
print(f"Strike grid    : {grid}")

ATM Strike     : 21700
Grid size      : 21 strikes
Strike range   : 20700 → 22700
Strike grid    : [20700, 20800, 20900, 21000, 21100, 21200, 21300, 21400, 21500, 21600, 21700, 21800, 21900, 22000, 22100, 22200, 22300, 22400, 22500, 22600, 22700]


In [24]:
# Narrower grid — 5 strikes each side, step=50
atm_50, grid_50 = md.get_atm_strikes(
    expiry="01FEB24",
    trade_date="01JAN24",
    n_strikes=5,
    step=50
)

print(f"ATM: {atm_50}")
print(f"Grid (step=50): {grid_50}")

ATM: 21700
Grid (step=50): [21450, 21500, 21550, 21600, 21650, 21700, 21750, 21800, 21850, 21900, 21950]


---
## 11. Spot Price Merge Validation
Verify that spot prices are correctly merged and contain no null values.

In [25]:
df_check = md.query_options(expiry="01FEB24", trade_date="01JAN24")

null_spot = df_check["spot_price"].isna().sum()
print(f"Null spot_price values : {null_spot}  ← should be 0")
print(f"Spot price range       : {df_check['spot_price'].min()} → {df_check['spot_price'].max()}")

# Show spot tracking against timestamps
df_check[["timestamp", "strike", "market_price", "spot_price"]].head(10)

Null spot_price values : 0  ← should be 0
Spot price range       : 21683.1 → 21831.8


,timestamp,strike,market_price,spot_price
0,2024-01-01 09:15:00,21700,271.60,21710.40
1,2024-01-01 09:15:00,23000,51.25,21710.40
2,2024-01-01 09:16:00,21750,410.15,21695.35
3,2024-01-01 09:16:00,21700,290.60,21695.35
4,2024-01-01 09:16:00,23000,44.85,21695.35
5,2024-01-01 09:16:00,22000,294.95,21695.35
6,2024-01-01 09:16:00,23300,24.30,21695.35
7,2024-01-01 09:17:00,21500,590.05,21709.55
8,2024-01-01 09:17:00,23150,32.75,21709.55
9,2024-01-01 09:18:00,21700,285.15,21701.60


In [26]:
# Verify spot updates minute-by-minute
spot_by_time = (
    df_check
    .drop_duplicates("timestamp")[["timestamp", "spot_price"]]
    .sort_values("timestamp")
    .head(15)
)
print(spot_by_time.to_string(index=False))

          timestamp  spot_price
2024-01-01 09:15:00    21710.40
2024-01-01 09:16:00    21695.35
2024-01-01 09:17:00    21709.55
2024-01-01 09:18:00    21701.60
2024-01-01 09:19:00    21693.75
2024-01-01 09:20:00    21687.90
2024-01-01 09:21:00    21694.70
2024-01-01 09:22:00    21693.35
2024-01-01 09:23:00    21692.55
2024-01-01 09:24:00    21695.65
2024-01-01 09:25:00    21697.05
2024-01-01 09:26:00    21699.65
2024-01-01 09:27:00    21703.05
2024-01-01 09:28:00    21699.65
2024-01-01 09:29:00    21700.80


---
## 12. Volatility Surface Snapshot
The primary deliverable for Team 2b. Builds a complete (expiry × strike) grid at a single timestamp — directly usable for implied volatility surface fitting.

In [27]:
surface = md.surface_snapshot(
    trade_date="01JAN24",
    timestamp="2024-01-01 10:00",
    n_expiries=6,
    n_strikes=10,
    step=100
)

print(f"Total rows in surface   : {len(surface)}")
print(f"Expiries included       : {sorted(surface['expiry_date'].unique())}")
print(f"Days-to-expiry range    : {surface['days_to_expiry'].min()} → {surface['days_to_expiry'].max()} days")
surface.head(10)

[NO DATA] Query returned 0 rows.
  Expiry: 26DEC24  |  Trade Date: 01JAN24
  Filters applied:
    strikes      = [20700, 20800, 20900, 21000, 21100, 21200, 21300, 21400, 21500, 21600, 21700, 21800, 21900, 22000, 22100, 22200, 22300, 22400, 22500, 22600, 22700]
    option_type  = None
    time window  = [2024-01-01 10:00, 2024-01-01 10:00]
    min_volume   = 0

  Suggestions:
    → Relax the min_volume filter (many rows have 0 volume).
    → Check available strikes: md.list_strikes('26DEC24', '01JAN24')
    → Check trading hours: NIFTY trades 09:15–15:30 IST.
Total rows in surface   : 175
Expiries included       : [datetime.date(2024, 1, 4), datetime.date(2024, 1, 11), datetime.date(2024, 1, 18), datetime.date(2024, 1, 25), datetime.date(2024, 2, 1)]
Days-to-expiry range    : 3 → 31 days


,timestamp,expiry_date,days_to_expiry,strike,option_type,open_price,high_price,low_price,close_price,market_price,volume,open_interest,spot_price
0,2024-01-01 10:00:00,2024-01-04,3,20700,C,1065.00,1065.00,1065.00,1065.00,1065.00,0,32000,21724.45
1,2024-01-01 10:00:00,2024-01-04,3,20700,P,1.95,2.00,1.95,1.95,1.95,24350,1628350,21724.45
2,2024-01-01 10:00:00,2024-01-04,3,20800,C,963.00,963.00,963.00,963.00,963.00,0,49650,21724.45
3,2024-01-01 10:00:00,2024-01-04,3,20800,P,2.40,2.40,2.30,2.30,2.30,19150,3116500,21724.45
4,2024-01-01 10:00:00,2024-01-04,3,20900,C,863.00,866.00,863.00,866.00,866.00,50,58650,21724.45
5,2024-01-01 10:00:00,2024-01-04,3,20900,P,3.15,3.15,3.05,3.05,3.05,70750,3495700,21724.45
6,2024-01-01 10:00:00,2024-01-04,3,21000,C,773.20,773.20,764.70,766.40,766.40,600,567000,21724.45
7,2024-01-01 10:00:00,2024-01-04,3,21000,P,4.00,4.00,3.90,3.90,3.90,31450,5219700,21724.45
8,2024-01-01 10:00:00,2024-01-04,3,21100,C,674.10,674.10,674.10,674.10,674.10,0,166650,21724.45
9,2024-01-01 10:00:00,2024-01-04,3,21100,P,5.15,5.15,5.05,5.05,5.05,42450,2126700,21724.45


In [28]:
# Summary: rows per expiry
print(surface.groupby("expiry_date")[["strike","market_price"]]
      .agg(n_strikes=("strike","count"),
           strike_min=("strike","min"),
           strike_max=("strike","max"),
           avg_price=("market_price","mean"))
      .round(2)
      .to_string())

             n_strikes  strike_min  strike_max  avg_price
expiry_date                                              
2024-01-04          42       20700       22700     284.76
2024-01-11          38       20700       22700     281.19
2024-01-18          30       20700       22700     207.55
2024-01-25          40       20700       22700     413.18
2024-02-01          25       21000       22700     328.41


In [29]:
# Calls-only surface (for standard Black-Scholes IV fitting)
surface_calls = md.surface_snapshot(
    trade_date="01JAN24",
    timestamp="2024-01-01 10:00",
    n_expiries=6,
    n_strikes=10,
    option_type="C",
    min_volume=0
)

print(f"Call surface rows: {len(surface_calls)}")
print(f"Option types     : {surface_calls['option_type'].unique()}")

[NO DATA] Query returned 0 rows.
  Expiry: 26DEC24  |  Trade Date: 01JAN24
  Filters applied:
    strikes      = [20700, 20800, 20900, 21000, 21100, 21200, 21300, 21400, 21500, 21600, 21700, 21800, 21900, 22000, 22100, 22200, 22300, 22400, 22500, 22600, 22700]
    option_type  = C
    time window  = [2024-01-01 10:00, 2024-01-01 10:00]
    min_volume   = 0

  Suggestions:
    → Relax the min_volume filter (many rows have 0 volume).
    → Check available strikes: md.list_strikes('26DEC24', '01JAN24')
    → Check trading hours: NIFTY trades 09:15–15:30 IST.
Call surface rows: 93
Option types     : ['C']


### How Pricing Team Uses This Output

```python
# Extract key columns directly for Black-Scholes
S     = surface_calls["spot_price"]       # Spot price
K     = surface_calls["strike"]            # Strike
T     = surface_calls["days_to_expiry"] / 365.0   # Time to expiry (years)
C_mkt = surface_calls["market_price"]     # Market option price

# Then pass to your IV solver:
# sigma = implied_vol(S, K, T, r, C_mkt, option_type='C')
```

---
## 13. Time Series Query (Multi-Day)
Track how a specific option's price evolves across multiple trading days.

In [30]:
# Get all trading days in January 2024
jan_days = md.list_trading_days(2024, "JAN")
print(f"January 2024 trading days ({len(jan_days)}): {jan_days}")

January 2024 trading days (21): ['01JAN24', '02JAN24', '03JAN24', '04JAN24', '05JAN24', '08JAN24', '09JAN24', '10JAN24', '11JAN24', '12JAN24', '15JAN24', '16JAN24', '17JAN24', '18JAN24', '19JAN24', '23JAN24', '24JAN24', '25JAN24', '29JAN24', '30JAN24', '31JAN24']


In [31]:
# Track the ATM call price at 10:00 AM each day
df_ts = md.query_time_series(
    expiry="01FEB24",
    trade_dates=jan_days[:5],          # first 5 days
    strikes=[21700],
    option_type="C",
    snapshot_time="10:00"
)

print(f"Rows returned: {len(df_ts)}")
df_ts[["trade_date", "timestamp", "strike", "option_type",
       "market_price", "spot_price", "days_to_expiry"]]

[SKIP] trade_date='03JAN24': [FILE NOT FOUND] Option file not found:
[SKIP] trade_date='04JAN24': [FILE NOT FOUND] Option file not found:
Rows returned: 3


,trade_date,timestamp,strike,option_type,market_price,spot_price,days_to_expiry
0,01JAN24,2024-01-01 10:00:00,21700,C,451.05,21724.45,31
1,02JAN24,2024-01-02 10:00:00,21700,C,406.00,21633.20,30
2,05JAN24,2024-01-05 10:00:00,21700,C,385.50,21741.20,27


---
## 14. Error Handling Demonstrations
The engine returns clear, actionable error messages. These examples show what happens when queries are malformed or data is unavailable.

### 14a. Invalid option_type

In [32]:
from api.marketdatav3 import InvalidParameter, FileNotAvailable, NoDataReturned

try:
    md.query_options(expiry="01FEB24", trade_date="01JAN24", option_type="CE")
except InvalidParameter as e:
    print("Caught InvalidParameter:")
    print(e)

Caught InvalidParameter:
[PARAMETER ERROR] option_type must be 'C' (Call) or 'P' (Put).
  Received: 'CE'


### 14b. Non-existent expiry

In [33]:
try:
    md.query_options(expiry="31DEC24", trade_date="01JAN24")
except FileNotAvailable as e:
    print("Caught FileNotAvailable:")
    print(e)

Caught FileNotAvailable:
[FILE NOT FOUND] Option file not found:
  Expiry    : 31DEC24
  TradeDate : 01JAN24
  Expected  : G:\.shortcut-targets-by-id\1f6XlJFCOVmETxGoJjD4O9WSmhB38IsQy\FinanceProject_LogicLabs\NiftyHistorical2024\raw_kaggle\2024\2024JAN\NIFTY-31DEC24-01JAN24.csv

  Possible causes:
    1. This expiry was not traded on 01JAN24.
       → Use md.list_expiries('01JAN24') to see what expiries are available.
    2. The date format may be wrong.
       → Required format: DDMMMYY (e.g. '01FEB24').
    3. Data for this period may not yet be loaded.
       → Contact the Data Pipeline Team.


### 14c. Bad date format

In [34]:
try:
    md.query_options(expiry="01FEB24", trade_date="2024-01-01")
except InvalidParameter as e:
    print("Caught InvalidParameter:")
    print(e)

Caught InvalidParameter:
[PARAMETER ERROR] trade_date '2024-01-01' is not a valid date.
  Required format: DDMMMYY  (e.g. '01JAN24', '15MAR25')


### 14d. Over-filtered query returns no data

In [35]:
# Very high min_volume — returns empty with a helpful message (no exception by default)
df_empty = md.query_options(
    expiry="01FEB24",
    trade_date="01JAN24",
    strikes=[21700],
    option_type="C",
    min_volume=999999
)
print(f"\nRows returned: {len(df_empty)}")

[NO DATA] Query returned 0 rows.
  Expiry: 01FEB24  |  Trade Date: 01JAN24
  Filters applied:
    strikes      = [21700]
    option_type  = C
    time window  = [None, None]
    min_volume   = 999999

  Suggestions:
    → Relax the min_volume filter (many rows have 0 volume).
    → Check available strikes: md.list_strikes('01FEB24', '01JAN24')
    → Check trading hours: NIFTY trades 09:15–15:30 IST.

Rows returned: 0


In [36]:
# With raise_if_empty=True — raises exception
try:
    md.query_options(
        expiry="01FEB24",
        trade_date="01JAN24",
        strikes=[21700],
        option_type="C",
        min_volume=999999,
        raise_if_empty=True
    )
except NoDataReturned as e:
    print("Caught NoDataReturned:")
    print(e)

Caught NoDataReturned:
[NO DATA] Query returned 0 rows.
  Expiry: 01FEB24  |  Trade Date: 01JAN24
  Filters applied:
    strikes      = [21700]
    option_type  = C
    time window  = [None, None]
    min_volume   = 999999

  Suggestions:
    → Relax the min_volume filter (many rows have 0 volume).
    → Check available strikes: md.list_strikes('01FEB24', '01JAN24')
    → Check trading hours: NIFTY trades 09:15–15:30 IST.


---
## 15. Performance Benchmarks

In [37]:
# Cold load (no cache)
md.clear_spot_cache()

t0 = time.time()
_ = md.query_options(expiry="01FEB24", trade_date="01JAN24")
t1 = time.time()
print(f"Cold load (spot not cached)  : {t1-t0:.3f}s")

# Warm load (spot is now cached)
t0 = time.time()
_ = md.query_options(expiry="01FEB24", trade_date="01JAN24", strikes=[21700])
t1 = time.time()
print(f"Warm load (spot cached)      : {t1-t0:.3f}s")

# Surface snapshot
t0 = time.time()
_ = md.surface_snapshot(trade_date="01JAN24", timestamp="2024-01-01 10:00", n_expiries=6)
t1 = time.time()
print(f"Surface snapshot (6 expiries): {t1-t0:.3f}s")

[CACHE] Spot cache cleared. (1 month(s) removed)
Cold load (spot not cached)  : 0.140s
Warm load (spot cached)      : 0.040s
[NO DATA] Query returned 0 rows.
  Expiry: 26DEC24  |  Trade Date: 01JAN24
  Filters applied:
    strikes      = [20700, 20800, 20900, 21000, 21100, 21200, 21300, 21400, 21500, 21600, 21700, 21800, 21900, 22000, 22100, 22200, 22300, 22400, 22500, 22600, 22700]
    option_type  = None
    time window  = [2024-01-01 10:00, 2024-01-01 10:00]
    min_volume   = 0

  Suggestions:
    → Relax the min_volume filter (many rows have 0 volume).
    → Check available strikes: md.list_strikes('26DEC24', '01JAN24')
    → Check trading hours: NIFTY trades 09:15–15:30 IST.
Surface snapshot (6 expiries): 0.858s


---
## 16. Cache Management

In [38]:
# Show what is currently cached
print("Cached months:", md.cache_status())

Cached months: {'2024JAN': 7875}


In [39]:
# Clear cache (useful for long-running sessions or after dataset update)
md.clear_spot_cache()
print("Cache after clearing:", md.cache_status())

[CACHE] Spot cache cleared. (1 month(s) removed)
Cache after clearing: {}


---
## Summary: Full API Reference

| Method | Purpose |
|--------|---------|
| `query_options(expiry, trade_date, ...)` | Core data fetch with all filters |
| `list_expiries(trade_date)` | Discover available expiries on a date |
| `list_strikes(expiry, trade_date)` | Discover available strikes |
| `list_trading_days(year, month)` | Discover trading days in a month |
| `get_atm_strikes(expiry, trade_date, ...)` | Generate ATM-centered strike grid |
| `query_time_series(expiry, trade_dates, ...)` | Multi-day evolution query |
| `surface_snapshot(trade_date, timestamp, ...)` | Full vol-surface input grid |
| `clear_spot_cache()` | Free memory / reset cache |
| `cache_status()` | Inspect what is in memory |